In [1]:
%run TAQplots.ipynb

part = []

whole = ['a', 'b', 'c' ,'d']

print(len(whole))

whole_ser = pd.Series(whole)
whole_ser[~whole_ser.isin(part)].to_list()

In [2]:
# the algorithm (CC.java) is taken from https://algs4.cs.princeton.edu/41graph/

class TAQ2TAQplots:
    def __init__(self, taq_df,
              row_id_col, # unique row id
              sec_col, # column with security code
              link_col, # orderno or some mark linking trades and quotes
              trdno_col,
              only_secs = []
             ):
        self.TAQplots_df = taq_df.copy()
        self.row_id_col = row_id_col
        self.sec_col = sec_col
        self.link_col = link_col
        self.trdno_col = trdno_col
        
        self.secs = pd.unique(taq_df[sec_col])
        if len(only_secs) > 0:
            only_secs_ser = pd.Series(only_secs)
            self.wrong_secs = only_secs_ser[~only_secs_ser.isin(self.secs)].to_list()
            if len(self.wrong_secs) > 0:
                self.TAQplots_df =pd.DataFrame()
                raise SystemExit(f"Wrong SECCODES: {self.wrong_secs}")
            else:
                self.secs = only_secs
        self.TAQplots_df = self.TAQplots_df[self.TAQplots_df.SECCODE.isin(self.secs)]
        
        self.vertex_count_dic = {k:v for (k, v) in zip(self.secs, [0 for i in range(len(self.secs))])}
        self.rowid2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.lnk2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.trdno2gkey_dic = {k:v for (k, v) in zip(self.secs, [{} for i in range(len(self.secs))])}
        self.edge_tpls = {k:v for (k, v) in zip(self.secs, [[] for i in range(len(self.secs))])}
        
        self.rows_to_edges()
        
        self.Gs = {k:v for (k, v) in zip(self.secs, [Graph(self.edge_tpls[sec],
                                                           self.vertex_count_dic[sec]) for sec in self.secs])}
        self.TAQpps = {k:v for (k, v) in zip(self.secs, [TAQplots(self.Gs[sec]) for sec in self.secs])}
        
        self.to_TAQplots_df()

    def sec_rowid_lnk2edges(self, rowid, sec, lnk, trdno): 
        # give 0, 1, 2, ... vertex names to all raw ids and "links" (eg ordernos)
        if rowid not in self.rowid2gkey_dic[sec]:
            self.rowid2gkey_dic[sec][rowid] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        if lnk not in self.lnk2gkey_dic[sec]:
            self.lnk2gkey_dic[sec][lnk] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        if trdno not in self.trdno2gkey_dic[sec]:
            self.trdno2gkey_dic[sec][trdno] = self.vertex_count_dic[sec]
            self.vertex_count_dic[sec] += 1
        self.edge_tpls[sec].append((self.rowid2gkey_dic[sec][rowid], 
                               self.lnk2gkey_dic[sec][lnk]))
        if pd.notna(trdno):
            self.edge_tpls[sec].append((self.rowid2gkey_dic[sec][rowid], 
                               self.trdno2gkey_dic[sec][trdno]))

    def rows_to_edges(self):
        [self.sec_rowid_lnk2edges(rowid, sec, lnk, trdno) 
         for rowid, sec, lnk, trdno in zip(self.TAQplots_df[self.row_id_col],
                                    self.TAQplots_df[self.sec_col],
                                    self.TAQplots_df[self.link_col],
                                    self.TAQplots_df[self.trdno_col])]
        
    def to_TAQplots_df(self):
        self.TAQplots_df['TAQPLOTNO'] = [self.TAQpps[sec].TAQplotnos[self.rowid2gkey_dic[sec][rowid]] 
                                    for (sec, rowid) in zip(self.TAQplots_df['SECCODE'], 
                                                            self.TAQplots_df['NO'])]

#OrderLog = 'OrderLog20020812'
OrderLog = 'OrderLog20071001'
OrderLog_df = pd.read_csv('./resources/' + OrderLog + '.txt')

OrderLog_df = OrderLog_df.astype({"NO":"int",
                                  "SECCODE":"string",
                                  "BUYSELL":"string",
                                  "TIME":"string",
                                  "ORDERNO":"int",
                                  "ACTION":"int",
                                  "PRICE":"float",
                                  "VOLUME":"int",
                                  "TRADENO":pd.Int64Dtype(),
                                  "TRADEPRICE":"float"
                                 })
OrderLog_df

TAQplots_df = TAQ2TAQplots(OrderLog_df, 'NO', 'SECCODE', 'ORDERNO', 'TRADENO', ['LKOH', 'EESR']).TAQplots_df
TAQplots_df.to_csv('./resources/' + OrderLog + '.csv')
TAQplots_df